# Introdução à Programação para Sensoriamento Remoto
ser347-2018
Aluno: Gabriel Moiano Cesar

Utilização da dimensão temporal para a interpolação de dados espúrios

O objetivo desse trabalho foi a implementação do método desenvolvido por Gigliotti et al. (2009) como rotina na linguagem Python. Essa metodologia permite uma melhor observação de fenômenos oceânicos com grande aplicabilidade em estudos em meso e macro-escala.


As etapas metodológicas a seguir foram utilizadas para atender os objetivos propostos.
Primeiramente foi feita a importação das bibliotecas utilizadas no programa:

In [ ]:
import numpy as np
import netCDF4 as nc
import glob
import os
from scipy.optimize import leastsq
import matplotlib.pyplot as plt

Definição das funções da série de dados:
Definição das características da série senoidal e dos dados espúrios:

In [ ]:
def jul2greg(dia):
    dias = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    if dia >= dias[0]:
        dd = 0
        for x, d in zip(range(1, 13), dias):
            dd += d
            if dd < dia:
                continue
            else:
                dg = dia - (dd - d)
                return x, dg
                break
    else:
        return 1, dia


os.chdir('F:\modis_sst')
names = glob.glob('*.nc')
names.sort()

mi = jul2greg(int(names[0][5:8]))[0]
mf = jul2greg(int(names[-1][5:8]))[0]

mifunc = mi * ((2 * np.pi) / 12)
mffunc = mf * ((2 * np.pi) / 12)

anoi = int(names[0][1:5])
anof = int(names[-1][1:5])

sst = []
for filename in names:
    print(filename)

    ds = nc.Dataset(filename)
    t = ds.variables['sst'][:]
    sst.append(t)

sst = np.array(sst)
sst_na = np.where(sst < -1000, np.nan, sst)
# sst_na2 = np.where(sst<-1000, np.nan, sst)

lenrad = np.linspace(mifunc, ((anof - anoi - 2) * 2 * np.pi) + mffunc, len(sst_na))

guess_phase = 0
guess_freq = 1
guess_amp = 1


Looping que realiza a função de ajuste:

In [ ]:
for l in range(sst_na.shape[1]):
    for c in range(sst_na.shape[2]):
        guess_mean = np.nanmean(sst_na[:, l, c])

        optimize_func = lambda x: x[0] * np.sin(x[1] * lenrad + x[2]) + x[3] - sst_na[:, l, c]
        est_amp, est_freq, est_phase, est_mean = leastsq(optimize_func, [guess_amp, guess_freq, guess_phase, guess_mean])[0]

        data_fit = est_amp * np.sin(est_freq * lenrad + est_phase) + est_mean
        na = np.argwhere(np.isnan(sst_na[:, l, c]))
        for i in na:
            sst_na[i[0], l, c] = data_fit[i[0]]

Plot da série (x,y) com e sem a substituição:

In [ ]:
plt.plot(sst_na)#f, ax = plt.subplots(2, 1)
ax[0].plot(t, sst_na[:, 90, 30], 'k-')
ax[0].plot(fine_t, data_fit2, 'r-', label='first guess')
ax[1].plot(t, sst_na[:, 90, 30], 'k-')
ax[1].plot(t, data_fit, 'b-', label='after fitting')
plt.legend()

Plot das imagens com os pixels substituídos:

In [ ]:
f, ax = plt.subplots(4, 3, figsize=(7.5, 10), sharex='col', sharey='row')
for x, a in zip(ax, np.arange((6), (18)).reshape(4, 3)):
    for i in range(3):
        x[i].imshow(sst_na[a[i]])

plt.show()